## This is the official training Demo of CellMix
* Use google colab pro+ (high RAM+GPU) for demo
* we use the 3090 GPU for the Experiments

## The code and Training process along with all record are private
* Our github page: https://github.com/sagizty/CellMix
* The datasets are publicly aviliable (mostly)


In [ ]:
# check GPU
!nvidia-smi

Tue Dec 13 04:31:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!date --date='+8 hour'  # CST time zone

Tue Dec 13 12:31:08 UTC 2022


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## create file-system enviroment
* mount your google drive first
* take the zipped dataset from the mounted google drive


In [ ]:
# clear colab path
!rm -rf /data
!rm -rf /home/Pathology_Experiment

# create path
!mkdir /home/Pathology_Experiment
!mkdir /home/Pathology_Experiment/runs
!mkdir /home/Pathology_Experiment/code
!mkdir /home/Pathology_Experiment/saved_models
!mkdir /home/Pathology_Experiment/imaging_results

!mkdir /data
!mkdir /data/Pathology_Experiment
!mkdir /data/Pathology_Experiment/dataset

print('Folder Tree Creation completed!')

# get latest code from Github pancreatic-cancer-diagnosis-tansformer page
!git clone https://github.com/sagizty/CellMix.git /home/Pathology_Experiment/code
print('code transfer from github completed!')

# copy runs if u want to compare
# !cp -r /content/drive/MyDrive/Pathology_Experiment/runs/* /home/Pathology_Experiment/runs
# print('tensorboard log transfer completed!')

# get the datasets
# by its zip
!cp /content/drive/MyDrive/dataset/ROSE_CLS.zip /data/Pathology_Experiment/dataset/
# !cp /content/drive/MyDrive/dataset/MARS_CLS.zip /data/Pathology_Experiment/dataset/
# !cp /content/drive/MyDrive/dataset/warwick_CLS.zip /data/Pathology_Experiment/dataset/
# !cp /content/drive/MyDrive/dataset/pRCC_CLS.zip /data/Pathology_Experiment/dataset/
# !cp /content/drive/MyDrive/dataset/GS_CLS.zip /data/Pathology_Experiment/dataset/

# unzip
!unzip -q /data/Pathology_Experiment/dataset/ROSE_CLS.zip -d /data/Pathology_Experiment/dataset/
# !unzip -q /data/Pathology_Experiment/dataset/MARS_CLS.zip -d /data/Pathology_Experiment/dataset/
# !unzip -q /data/Pathology_Experiment/dataset/warwick_CLS.zip -d /data/Pathology_Experiment/dataset/
# !unzip -q /data/Pathology_Experiment/dataset/pRCC_CLS.zip -d /data/Pathology_Experiment/dataset/
# !unzip -q /data/Pathology_Experiment/dataset/GS_CLS.zip -d /data/Pathology_Experiment/dataset/

# alter the path
!rm -rf /data/Pathology_Experiment/dataset/ROSE_CLS.zip
# !rm -rf /data/Pathology_Experiment/dataset/MARS_CLS.zip
# !rm -rf /data/Pathology_Experiment/dataset/warwick_CLS.zip
# !rm -rf /data/Pathology_Experiment/dataset/pRCC_CLS.zip
# !rm -rf /data/Pathology_Experiment/dataset/GS_CLS.zip
print('data transfer completed!')

Folder Tree Creation completed!
Cloning into '/home/Pathology_Experiment/code'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 119 (delta 27), reused 0 (delta 0), pack-reused 72
Receiving objects: 100% (119/119), 102.55 KiB | 20.51 MiB/s, done.
Resolving deltas: 100% (51/51), done.
code transfer from github completed!
data transfer completed!


## Arrange the working enviorment

In [ ]:
!pip install -q torch==1.10.0+cu111 torchvision==0.11.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████▌                   | 834.1 MB 1.2 MB/s eta 0:18:38tcmalloc: large alloc 1147494400 bytes == 0x3a1fc000 @  0x7f06f45ca615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |███████████████▉                | 1055.7 MB 130.8 MB/s eta 0:00:09tcmalloc: large alloc 1434370048 bytes == 0x7e852000 @  0x7f06f45ca615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████            | 1336.2 MB 1.2 MB/s eta 0:10:54tcmalloc: large alloc 1792966656 bytes == 0x3684000 @  0x7f06f45ca615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0

In [ ]:
# change working dir
import os
os.chdir("/home/Pathology_Experiment/code")
!pwd

# get packages
!pip install tensorboardX
!pip install timm
!pip install notifyemail
!pip install ttach

!pip freeze>requirements.txt
!cp requirements.txt ../runs

/home/Pathology_Experiment/code
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 30.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 44.5 MB/s 
     |████████████████████████████████| 182 kB 89.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python --version

Python 3.8.16


In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh                         2.3.3
branca

## Start Training
* by command line
* use argparse to set down hyper-parameter

* ViT baseline and ViT + CellMix are trained and compared here.

# CLS counterparts

* Train

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --patch_strategy loss_hold --ratio_strategy loss_hold --enable_tensorboard --edge_size 384 --data_augmentation_mode 0 --lr 0.00001 --lrf 0.25 --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/runs

class_names: ['Negative', 'Positive']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, PromptTuning=None, PromptUnFreeze=False, Prompt_Token_num=20, Prompt_state_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=8, check_minibatch=None, cls_token_off=False, data_augmentation_mode=0, dataroot='/data/Pathology_Experiment/dataset/ROSE_CLS', draw_root='/home/Pathology_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, fix_patch_size=None, fix_position_ratio=None, gpu_idx=-1, intake_epochs=0, linearprobing=False, loss_drive_threshold=4.0, lr=1e-05, lrf=0.25, model_idx='ViT_384_401_PT_lf25_b8_ROSE_CLS', model_path='/home/Pathology_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True,

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --enable_attention_check --edge_size 384 --data_augmentation_mode 0 --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/runs

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

# CellMix

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --augmentation_name CellMix-Group --patch_strategy loss_hold --ratio_strategy loss_hold --enable_tensorboard --edge_size 384 --data_augmentation_mode 0 --lr 0.00001 --lrf 0.25 --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/runs

class_names: ['Negative', 'Positive']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, PromptTuning=None, PromptUnFreeze=False, Prompt_Token_num=20, Prompt_state_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name='CellMix-Group', backbone_PT_off=False, batch_size=8, check_minibatch=None, cls_token_off=False, data_augmentation_mode=0, dataroot='/data/Pathology_Experiment/dataset/ROSE_CLS', draw_root='/home/Pathology_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, fix_patch_size=None, fix_position_ratio=None, gpu_idx=-1, intake_epochs=0, linearprobing=False, loss_drive_threshold=4.0, lr=1e-05, lrf=0.25, model_idx='ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS', model_path='/home/Pathology_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --enable_attention_check --edge_size 384 --data_augmentation_mode 0 --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/runs

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

# visulizations

check cam on shuffled samples, from patch size 16 to 192. the fixposition ratio is 0.5

--fix_position_ratio 0.5 stands for using 50% of patches fixed, the remaining shuffled

--puzzle_patch_size 16 stands for using the patch size of 16

--batch_size 4 stands for using the batch size of 4 images

--enable_attention_check stands for using Grad-CAM to visulize the attention

--shuffle_dataloader stands for using the shuffled test dataloader instead of test dataloader with same image taking order

--draw_root the output path to store the images 

## ViT

--fix_position_ratio 0.5 --puzzle_patch_size 16 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 16 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 32 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 32 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 48 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 48 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 64 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 64 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 96 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 96 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 128 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 128 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 192 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 192 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

## CellMix Trained ViT

--fix_position_ratio 0.5 --puzzle_patch_size 16 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 16 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 32 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 32 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 48 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 48 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 64 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 64 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 96 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 96 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 128 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 128 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

--fix_position_ratio 0.5 --puzzle_patch_size 192 --batch_size 4 --enable_attention_check --shuffle_dataloader

In [ ]:
!python CellMix_test.py --model_idx ViT_384_401_PT_lf25_b8_ROSE_CellMix_Group_CLS --gpu_idx 0 --enable_visualize_check --edge_size 384 --data_augmentation_mode 0 --augmentation_name CellMix-Group --fix_position_ratio 0.5 --puzzle_patch_size 192 --batch_size 4 --enable_attention_check --shuffle_dataloader --dataroot /data/Pathology_Experiment/dataset/ROSE_CLS --model_path /home/Pathology_Experiment/saved_models --draw_root /home/Pathology_Experiment/imaging_results

class_names: ['Negative', 'Positive']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'gcvit_base',
 'gcvit_small',
 'gcvit_tiny',
 'gcvit_xtiny',
 'gcvit_xxtiny',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_256d',
 'levit_384',
 'maxvit_base_224',
 'maxvit_large_224',
 'maxvit_nano_rw_256',
 'maxvit_pico_rw_256',
 'maxvit_rmlp_nano_rw_256',
 'maxvit_rmlp_pico_rw_256',
 'maxvit_rmlp_small_rw_224',
 'maxvit_rmlp_small_rw_256',
 'maxvit_rmlp_tiny_rw_256',
 'maxvit_small_224',
 'maxvit_tiny_224',
 'maxvit_tiny_pm_256',
 'maxvit_tiny_rw_224',
 'maxvit_tiny_rw_256',
 'maxvit_xlarge_224',
 'maxxvit_rmlp_nano_rw_256',
 'maxxvit_rmlp_small_rw_256',
 'maxxvit_rmlp_tiny_rw_256',
 'mobilevit_s',
 'mobilevit_xs',
 'mobi

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/home/Pathology_Experiment/runs'

<IPython.core.display.Javascript object>

# Synchronize files

In [ ]:
# # copy tensorboard runs
!/bin/cp -rf /home/Pathology_Experiment/runs/*  /content/drive/MyDrive/Pathology_Experiment/runs/
print('runs copy completed!')
# copy the traind models
!/bin/cp -rf /home/Pathology_Experiment/saved_models/* /content/drive/MyDrive/Pathology_Experiment/saved_models/
print('models copy completed!')
# copy the imaging_results
!/bin/cp -rf /home/Pathology_Experiment/imaging_results/* /content/drive/MyDrive/Pathology_Experiment/imaging_results/
print('imaging_results copy completed!')

runs copy completed!
models copy completed!
imaging_results copy completed!


In [ ]:
!date --date='+8 hour'  # CST time zone

Tue Dec 13 13:31:45 UTC 2022
